# Rating a Single-Stage Vapor Compression Machine
---

**Importing libraries and notebook preparation**

In [1]:
from pathlib import Path

from hvac import Quantity
from hvac.fluids import Fluid, HumidAir, CoolPropWarning
from hvac.vapor_compression import VariableSpeedCompressor
from hvac.vapor_compression import SS_VCM
from hvac.heat_exchanger.recuperator.fintube import PFT_CF_AE
from hvac.heat_exchanger.recuperator.fintube import PFT_CF_AC

import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=CoolPropWarning)

Q_ = Quantity

## Configuration of the Vapor Compression Machine

A single-stage vapor compression machine is composed of a compressor, an evaporator, a condenser, and an expansion device. Air with a given state and mass flow rate enters the evaporator. Also at the condenser, air enters the condenser with a given state and mass flow rate. The expansion device regulates the mass flow rate of refrigerant entering the evaporator so that the set degree of refrigerant superheating at the evaporator exit is maintained. The compressor runs at a given compressor speed.

The question is what the steady-state performance of the vapor compression machine will be under these given operating conditions.

To configure a single-stage vapor compression machine, we first need to instantiate the models of the compressor, the evaporator and the condenser.

**Compressor**<br>
After rating the evaporator and the condenser (see notebooks *ex02_air_evaporator.ipynb* and *ex03_air_condenser.ipynb*) we keep the originally selected variable speed compressor DANFOSS VTZ038-G with refrigerant R134a. The coefficients of the polynomial equations describing the performance quantities of this compressor were saved in a csv-file.

First, we define the refrigerant.

In [2]:
R134a = Fluid('R134a')

Now we can instantiate the compressor model.

In [3]:
compressor = VariableSpeedCompressor(
    coeff_file=Path("compressor_data/VTZ038-G_R134a.csv"),
    refrigerant=R134a,
    dT_sh=Q_(10, 'K'),
    dT_sc=Q_(5, 'K'),
    units={'m_dot': 'kg / hr', 'n': 'rps'}
)

**Evaporator**<br>
We use the same air evaporator that we previously have rated.

In [4]:
evaporator = PFT_CF_AE(
    W_fro=Q_(0.731, 'm'),
    H_fro=Q_(0.244, 'm'),
    N_rows=6,
    S_trv=Q_(25.4, 'mm'),
    S_lon=Q_(22.0, 'mm'),
    D_int=Q_(8.422, 'mm'),
    D_ext=Q_(10.2, 'mm'),
    t_fin=Q_(0.3302, 'mm'),
    N_fin=1 / Q_(3.175, 'mm')
)

**Condenser**<br>
We use the same air condenser that we previously have rated.

In [5]:
condenser = PFT_CF_AC(
    W_fro=Q_(1.003, 'm'),
    H_fro=Q_(0.334, 'm'),
    N_rows=5,
    S_trv=Q_(25.4, 'mm'),
    S_lon=Q_(22.0, 'mm'),
    D_int=Q_(8.422, 'mm'),
    D_ext=Q_(10.2, 'mm'),
    t_fin=Q_(0.3302, 'mm'),
    N_fin=1 / Q_(3.175, 'mm')
)

**Vapor compression machine**<br>
Instantiation of the vapor compression machine is straightforward. First, we pass the evaporator, the condenser, and the compressor model to the constructor. Then, we specify the refrigerant and the degree of refrigerant superheating that is set on the expansion device:

In [6]:
machine = SS_VCM(
    evaporator, condenser, compressor,
    dT_sh=Q_(10, 'K'),
    n_cmp_min=Q_(2100, 'rpm'),
    n_cmp_max=Q_(5400, 'rpm')
)

## Specify Known Operating Conditions and Determine Steady-State Machine Operation

The following inputs will unambiguously determine the operation of the vapor compression machine:
- the mass flow rate of air through the evaporator
- the mass flow rate of air through the condenser
- the state of air at the evaporator inlet
- the state of air at the condenser inlet
- the compressor speed (only in case of a variable speed compressor)

To determine the steady-state performance of the machine, an iterative solving technique is used that is based on the method explained in Chapter 14 of the book *Stoecker, W. F., & Jones, J. W. (1982). Refrigeration and air conditioning. McGraw-Hill International Editions, Mechanical Technology Series.*
The implemented rating routine uses a minimization algorithm (`scipy.optimize.minimize`) to determine the evaporation and condensation temperature for which the deviation is minimal between the refrigerant mass flow rate according to the compressor model, and the refrigerant mass flow rate according to the evaporator model (i.e., the mass flow rate the expansion device lets through to maintain the set degree of refrigerant superheating at the outlet of the evaporator). The routine needs initial guesses for the evaporation and condensing temperature to start with. If the user does not provide these initial guesses, the rating routine can guess initial values itself. However, it may happen that no solution will be found with one set of initial guesses, but that a solution can still be found with another set.

To determine the steady-state performance of the vapor compression machine in this example, we will use the operating conditions that we have used so far. The air mass flow rate through the condenser is taken from *ex03_air_condenser.ipynb*. The compressor speed is the speed returned by the compressor selection program when we rated the air condenser.

In [7]:
output = machine.rate(
    evp_air_in=HumidAir(Tdb=Q_(24.0, 'degC'), RH=Q_(50, 'pct')),
    evp_air_m_dot=Q_(1500, 'kg / hr'),
    cnd_air_in=HumidAir(Tdb=Q_(35.0, 'degC'), RH=Q_(30, 'pct')),
    cnd_air_m_dot=Q_(2694.293, 'kg / hr'),
    n_cmp=Q_(4407, 'rpm'),
    T_evp_ini=Q_(10, 'degC'),
    T_cnd_ini=Q_(50, 'degC')
)

[2564 | hvac.vapor_compression.machine | INFO] Starting machine operation analysis with...
[2564 | hvac.vapor_compression.machine | INFO] n_cmp = 4407 rpm
[2564 | hvac.vapor_compression.machine | INFO] evp_air_in: 24.00 °C, 50 %
[2564 | hvac.vapor_compression.machine | INFO] evp_air_m_dot: 1500.00 kg/h
[2564 | hvac.vapor_compression.machine | INFO] cnd_air_in: 35.00 °C, 30 %
[2564 | hvac.vapor_compression.machine | INFO] cnd_air_m_dot: 2694.29 kg/h
[2564 | hvac.vapor_compression.machine | INFO] Iteration 1: Try with: T_evp = 10.000 °C, T_cnd = 50.000 °C
[2564 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant mass flow rate from compressor = 144.832 kg/h
[2564 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant entering condenser with T = 84.454 °C, P = 13.179 bar
[2564 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant leaving condenser with T = 47.023 °C, P = 13.179 bar
[2564 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant

In [8]:
print(output.to_text())

evp_air_m_dot = 1500.000 kg/h
cnd_air_m_dot = 2694.293 kg/h
evp_air_in = 24.000 °C DB, 9.340 g/kg AH (50 % RH)
cnd_air_in = 35.000 °C DB, 10.590 g/kg AH (30 % RH)
n_cmp = 4407.000 rpm
dT_sh = 10.000 K
evp_air_out = 13.133 °C DB, 8.130 g/kg AH (86 % RH)
cnd_air_out = 45.242 °C DB, 10.590 g/kg AH (17 % RH)
evp_Q_dot = 5.909 kW
cnd_Q_dot = 7.870 kW
cmp_W_dot = 1.969 kW
rfg_m_dot = 144.832 kg/h
COP = 3.996 frac
EER = 3.000 frac
evp_eps = 0.7500 frac
cnd_eps = 0.5553 frac
T_evp = 10.000 °C
P_evp = 4.146 bar
T_cnd = 50.000 °C
P_cnd = 13.179 bar
dT_sc = 2.977 K
suction_gas = 20.000 °C, 413.669 kJ/kg, 4.146 bar
discharge_gas = 84.454 °C, 462.619 kJ/kg, 13.179 bar
liquid = 47.023 °C, 266.999 kJ/kg, 13.179 bar
mixture = 10.000 °C, 266.999 kJ/kg, 4.146 bar, 28 %
evp_air_dP = 45.896 Pa
cnd_air_dP = 39.740 Pa

